In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import itertools
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize

def initialize_NN(layers):
    """
    Structure and initilization of the NN.
    :param layers: A list of neurons in each layer(including the input and output layers)
    :returns: The intialized weights and biases of the required choice of the hidden layers
    """
    weights = []
    biases = []
    num_layers = len(layers) 
#     with tf.name_scope(scope):
    for l in range(0,num_layers-1):
        W = xavier_initialisation(size=[layers[l], layers[l+1]])
        b = tf.Variable(tf.zeros([1,layers[l+1]]))
        weights.append(W)
        biases.append(b)        
    return weights, biases
    
def xavier_initialisation(size):
    """
    Initializes the NN.
    :param size: The dimensions
    :returns: The intialized weights or biases for one specific hidden layer
    """
    in_dim = size[0]
    out_dim = size[1]        
    xavier_stddev = np.sqrt(20./(in_dim + out_dim))
    a = tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev))
    return a

def forward(x, weights, biases):
    """
    Forward propogation of the Neural Network
    :param : Time and position of the particle and the parameters of the network
    :returns: Velocity and pressure
    """
    num_layers = len(weights) 
    H = x
    for l in range(num_layers):
        W = weights[l]
        b = biases[l]
        H = tf.add(tf.matmul(H, W), b)
        if l == num_layers - 1:
            H = tf.identity(H, name='class')
        else:
            H = tf.tanh(H)
    return H

data = pd.read_csv("Concrete_Data_missing.csv", delimiter=',', index_col=0)
train_data_x = data.loc[:800, data.columns !='Concrete compressive strength(MPa, megapascals) ']
train_data_y = data.loc[:800, ['Concrete compressive strength(MPa, megapascals) ']]

test_data_x = data.loc[800:, data.columns !='Concrete compressive strength(MPa, megapascals) '].values
test_data_y = data.loc[800:, ['Concrete compressive strength(MPa, megapascals) ']].values

iterable = list(np.arange(8))
pos_list = list(itertools.combinations(iterable, 6))
pos_list = pos_list

In [2]:
class standardscaler():
    def fit(self, x_train):
        self.mean = np.mean(x_train)
        self.scale = np.std(x_train)
        
    def transform(self, x_test):
        return (x_test-self.mean)/self.scale
    
    def inversetransform(self, x_test):
        return x_test*self.scale+self.mean
    

In [3]:
vel_layers = [6, 100, 100, 100, 100, 1]

for j in range(len(pos_list)):
    
    tf.reset_default_graph()
    with tf.name_scope('scope_%0i' %j):

        vel_weights, vel_biases = initialize_NN(vel_layers)

        data_x_cur =  train_data_x.iloc[:,map(int, pos_list[j])]
        df_train = pd.concat([data_x_cur, train_data_y], axis=1)
        train_index = np.random.randint(800, size=800)
        oob_index = np.setdiff1d(np.arange(800),train_index)
        df_train = df_train.iloc[train_index]
        df_val = df_train.iloc[oob_index]
        df_train = df_train.dropna().values
        df_val = df_val.dropna().values

        x_train = df_train[:,:-1]
        y_train = df_train[:,-1][:,None]

        x_val = df_val[:,:-1]
        y_val = df_val[:,-1][:,None]

        scaler = standardscaler()
        scaler.fit(x_train)
        epochs=200
        x_pl = tf.placeholder('float',[None,6], name='input_placeholder')
        x_test_pl = tf.placeholder('float',[None,8], name='test_placeholder')
        y_pl = tf.placeholder('float',[None,1])

        cost = tf.losses.mean_squared_error(labels=y_pl, predictions=forward(x_pl, vel_weights, vel_biases))
        optimizer = tf.train.AdamOptimizer().minimize(cost)
        batch_size = 50

        train_accuracy = np.zeros(epochs)
        val_accuracy = np.zeros(epochs)

        with tf.Session() as sess:
            x_test = tf.stack([x_test_pl[:,a] for a in pos_list[j]], axis=1)

            prediction = tf.identity(forward(scaler.transform(x_test), vel_weights, vel_biases), 'predict')
            sess.run(tf.variables_initializer(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='scope_%0i' %j)))
            for epoch in range(epochs):
                epoch_loss = 0
                for i in range(int(x_train.shape[0]/batch_size)):
                    epoch_x,epoch_y = x_train[i*batch_size:(i+1)*batch_size,:], y_train[i*batch_size:(i+1)*batch_size,:]
                    sess.run(optimizer, feed_dict = {x_pl:scaler.transform(epoch_x),y_pl:epoch_y})
            error = tf.identity(sess.run(cost, feed_dict = {x_pl:scaler.transform(x_val),y_pl:y_val}), 'val_error')
            saver = tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='scope_%0i' %j))
            saver.save(sess, './my_test_model%i' %j)


In [4]:
error = []
result = []
for i in range(len(pos_list)):
    tf.reset_default_graph()
    with tf.Session() as sess:    
        saver = tf.train.import_meta_graph('my_test_model%0i.meta' %i)
        saver.restore(sess, './my_test_model%i' %i)
        result.append(sess.run('scope_%i/predict:0' %i, feed_dict={'scope_%i/test_placeholder:0' %i: test_data_x}))
        error.append(sess.run('scope_%i/val_error:0'%i))
accuracy = np.max(error)-error
normalized = normalize(accuracy[:,np.newaxis], axis=0, norm='l1').ravel()
final_prediction = np.sum(np.concatenate([result[i] for i in range(len(result))], axis=1)*normalized, axis=1)


INFO:tensorflow:Restoring parameters from ./my_test_model0
INFO:tensorflow:Restoring parameters from ./my_test_model1
INFO:tensorflow:Restoring parameters from ./my_test_model2
INFO:tensorflow:Restoring parameters from ./my_test_model3
INFO:tensorflow:Restoring parameters from ./my_test_model4
INFO:tensorflow:Restoring parameters from ./my_test_model5
INFO:tensorflow:Restoring parameters from ./my_test_model6
INFO:tensorflow:Restoring parameters from ./my_test_model7
INFO:tensorflow:Restoring parameters from ./my_test_model8
INFO:tensorflow:Restoring parameters from ./my_test_model9
INFO:tensorflow:Restoring parameters from ./my_test_model10
INFO:tensorflow:Restoring parameters from ./my_test_model11
INFO:tensorflow:Restoring parameters from ./my_test_model12
INFO:tensorflow:Restoring parameters from ./my_test_model13
INFO:tensorflow:Restoring parameters from ./my_test_model14
INFO:tensorflow:Restoring parameters from ./my_test_model15
INFO:tensorflow:Restoring parameters from ./my_tes

In [5]:
print(mean_squared_error(final_prediction, test_data_y))


29.827807874850556
